In [14]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from sinkhorn import sinkhorn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Initialization of variables:

In [15]:
# Points:
n=10
x = np.linspace(0,1,n)
y = np.linspace(0,1,n)
x=x[:,np.newaxis]
y=y[:,np.newaxis]
# Cost:
C = (x-y.T)**2
# entropy factor:
eta = 1
# Kernel:
Kmat = np.exp(-eta*C)
def K(v):
    return Kmat@v
def Kt(v):
    return Kmat.T@v
# Target marginals:
p = np.random.rand(n,1)
p = p / np.sum(p)
q = np.random.rand(n,1)
p = q / np.sum(q)
# tolerance:
delta = 1e-2

### Sinkhorn projection computation:

In [19]:
u,v,W = sinkhorn(K,Kt,p,q,delta)
print(u.T)
print(v.T)
print(W)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[7.11563277e+307 4.69476300e+307 1.31956513e+307 4.41526062e+307
  3.11995198e+307 1.07454503e+307 4.93031171e+307 2.93261498e+307
  2.47144272e+306 4.93077073e+307]]
[[nan]]


/var/folders/4p/dkkns1r90jngnrbymzmfpw840000gn/T/ipykernel_99937/3337660702.py:14: RuntimeWarning: overflow encountered in matmul
  return Kmat@v
/Users/benoitmuller/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/Users/benoitmuller/Documents/EPFL/low_rank/Low_Rank_Approximation_Techniques_Project/sinkhorn.py:21: RuntimeWarning: invalid value encountered in multiply
  while np.sum(np.abs(u*(K(v) - p))) + np.sum(np.abs(v*(Kt(u) - q))) >= delta/2:
/Users/benoitmuller/Documents/EPFL/low_rank/Low_Rank_Approximation_Techniques_Project/sinkhorn.py:27: RuntimeWarning: divide by zero encountered in log
  W = np.log(u).T @ (u*K(v)) + np.log(v).T @ (v*Kt(u))
/Users/benoitmuller/Documents/EPFL/low_rank/Low_Rank_Approximation_Techniques_Project/sinkhorn.py:27: RuntimeWarning: invalid value encountered in multiply
  W = np.log(u).T @ (u*K(v)) + np.log(v).T @ (v*Kt(u))
